[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task3_dmrf_OOP.ipynb)

# Задание
Реализовать с помощью объектно-ориентированного подхода предыдущие 2 задания. Создайте для каждой из задач отдельный класс, который позволяет ее решить.

* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe, а также в excel, pickle, БД

* Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных

*  Сделать визуализацию для мини-исследования рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.



In [406]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import os
import wget
from tqdm import tqdm

%matplotlib inline

## Определение класса, который получает ID дома

In [407]:
class DomIdLoader:
    """Выгружает ID жилых комплексов с сайта наш.дом.рф. Для работы нужен tqdm"""
    def __init__(self):
        self.limit_ = 1000
        self.url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
        self.objects_data = []

    def get_ids(self, amount_1k='all'):
        """
        Начинает загрузку ID объектов. По умолчанию выгружает 33000 объектов.
        При передаче аргумента важно посмотреть на сайте количество записей в списке, чтобы избежать лишних итераций.
        :param amount_1k: Если в метод передать числовой аргумент, то выгрузится соответствующее количество ID в тысячах штук.
        Если с сайта будут получены дублирующие ID, то они будут удалены и сохранятся только уникальные,
        при этом количество будет меньше, т.к. дубли убираются.
        :return: По результатам выводится сообщение об успешности
        """
        if amount_1k == 'all':
            for i in tqdm(range(33)):
                paramz = {
                    'offset': self.limit_ * i,
                    'limit': self.limit_,
                    'sortField':'devId.devShortCleanNm',
                    'sortType':'asc',
                }
                res = requests.get(self.url, params=paramz)

                self.objects_data += res.json().get('data').get('list')
            self.objects_data = set([i.get('objId') for i in self.objects_data if type(i.get('objId')) is int])
            self.objects_data = sorted(list(self.objects_data))
        else:
            for i in tqdm(range(amount_1k)):
                paramz = {
                    'offset': self.limit_ * i,
                    'limit': self.limit_,
                    'sortField':'devId.devShortCleanNm',
                    'sortType':'asc',
                }
                res = requests.get(self.url, params=paramz)

                self.objects_data += res.json().get('data').get('list')
            self.objects_data = set([i.get('objId') for i in self.objects_data if type(i.get('objId')) is int])
            self.objects_data = sorted(list(self.objects_data))
            return print('Данные загружены')

    def show_ids(self):
        """Выводит на печать все загруженные ID"""
        print(*self.objects_data)

    def list_id(self, amount='all'):
        """
        Выводит список загруженных ID.
        :param amount: Если передать внутрь метода число, то на выходе будет список, длинною в заданое значение
        :return: возвращает список
        """
        if amount == 'all':
            return self.objects_data
        else:
            return self.objects_data[:amount]


# Определение класса, который выгружает данные по жилым комплексам

In [408]:
class ObjectInfoExtractor:
    """
    Выгружает данные по объектам. Может вывести предобаботанные данные списком,
    необработанные данные списком и вывести данные в формате датафрейма. Для работы нужен tqdm
    """
    def __init__(self):
        self.objects_info = []

    def load_data(self, ids, amount='all'):
        """
        Скачивает информацию по объетам.
        :param ids: подается список ID или единственное значение в формате списка
        :param amount: по умолчанию загружается информация по всем объектам.
        Указав целое число можно ограничить количество загружаемых объектов.
        :return: по завершению загрузки выводится сообщение о готовности.
        """
        if amount == 'all':
            for i in tqdm(ids):
                res = requests.get(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}')
                self.objects_info += [res.json()]
        else:
            for i in tqdm(ids[:amount]):
                res = requests.get(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}')
                self.objects_info += [res.json()]
        return print('Данные загружены')

    def df_converter(self):
        """
        Выводит данные, готовые к загрузке в датафрейм с помощью pd.json_normalize()
        :return: возвращает список словарей
        """
        return [i.get('data') for i in self.objects_info]

    def not_converted(self):
        """
        Выводит неконвертированные данные
        :return: возвращает список
        """
        return self.objects_info

    def to_df(self):
        """
        Выводит данные в формате датафрейма
        """
        return pd.json_normalize([i.get('data') for i in self.objects_info])

# Определение класса, который будет сохранять датафрейм в разные форматы и загружать фотографии

In [460]:
class Saver:
    """
    Класс сохраняет предварительно очищенный от нулевых строк датафрейм в разные форматы.
    На вход необходимо подать датафрейм, полученный при выгрузке с сайта наш.дом.рф.
    Также, реализован метод сохранения фотографий. Для работы нужны sqlite3, os, wget, tqdm
    """
    def __init__(self, data):
        self.data = data.dropna(how='all')

    def save_csv(self):
        """
        Сохраняем данные в csv
        :return: вывод сообщения о сохранении данных
        """
        # пробуем создать директорию data, если ее нет, если есть, то ничего не делаем
        try:
            os.mkdir(os.path.join('data'))
        except FileExistsError:
            pass
        self.data.to_csv(os.path.join('data', 'nash_dom_df.csv'))
        return print("Данные сохранены")

    def save_xl(self):
        """
        Сохраняем данные в excel
        :return: вывод сообщения о сохранении данных
        """
        # пробуем создать директорию data, если ее нет, если есть, то ничего не делаем
        try:
            os.mkdir(os.path.join('data'))
        except FileExistsError:
            pass
        self.data.to_excel(os.path.join('data', 'nash_dom_df.xlsx'))
        return print("Данные сохранены")

    def save_sql(self):
        """
        Сохраняем данные в БД SQL
        :return: вывод сообщения о сохранении данных
        """
        copy_df = self.data.copy()
        for i in copy_df:
            if type(copy_df[i][0]) is list:
                copy_df[i] = copy_df[i].apply(str)
        copy_df = copy_df.drop(['metro.colors'], axis=1)
        try:
            os.mkdir(os.path.join('data'))
        except FileExistsError:
            pass
        conn = sqlite3.connect(os.path.join('data', 'nash_dom_df.db'))
        copy_df.to_sql('real_estate', conn, if_exists='replace', index=False)
        conn.commit()
        conn.close()
        return print("Данные сохранены")

    def save_pics(self, startswith=0, end=None):
        """
        Осуществляет выгрузку фотографий по ЖК из датафрейма.
        :param startswith: указывается индекс строки, с которого начинается загрузка фото. По умолчанию значение 0
        :param end: указывается индекс строки, до которого будет продолжаться загрузка фото. По умолчанию выгрузка осуществляется до конца DF
        :return: после окончания загрузки выводится сообщение об успешности
        """
        pics_df = self.data[['id', 'photoRenderDTO']].copy()
        pics_df.reset_index(drop=True, inplace=True)
        pics_df.id = pics_df.id.apply(int)
        def pics_url(x):
            pics_url = []
            for i in x:
                pics_url += [i.get('objRenderPhotoUrl')]
            return pics_url
        pics_df.photoRenderDTO = pics_df.photoRenderDTO.apply(pics_url)
        start_dir = os.getcwd()
        try:
            os.mkdir(os.path.join('data'))
        except FileExistsError:
            pass
        try:
            os.mkdir(os.path.join('data', 'pics'))
        except FileExistsError:
            pass
        if end is None:
            end = len(pics_df.photoRenderDTO)
        for i in tqdm(range(startswith, end)):
            if pics_df.photoRenderDTO[i]:
                try:
                    os.mkdir(f'data/pics/{pics_df.id.iloc[i]}')
                except FileExistsError:
                    pass
                os.chdir(os.path.join('data', 'pics', f'{pics_df.id.iloc[i]}'))
                for link in pics_df.photoRenderDTO[i]:
                    if str(link).startswith('htt'):
                        wget.download(link)
                os.chdir(start_dir)
        return print('Фото выгружены')


In [410]:
#class Visualizer:
    #def __init__(self,data):
        #...

    #def make_boxplot(self, ):
        #...

    #def make_heatmap(self):
        #...

In [411]:
objid = DomIdLoader()

In [412]:
objid.get_ids()

100%|██████████| 33/33 [06:12<00:00, 11.28s/it]


In [413]:
len(objid.list_id())

31416

In [414]:
dwld = ObjectInfoExtractor()

In [415]:
dwld.load_data(objid.list_id())

100%|██████████| 31416/31416 [1:35:12<00:00,  5.50it/s]  

Данные загружены


In [438]:
df = dwld.to_df()

In [461]:
save = Saver(df)

In [440]:
save.save_sql()
save.save_csv()
save.save_xl()

Данные сохранены
Данные сохранены
Данные сохранены


In [462]:
save.save_pics(startswith=5, end=60)

100%|██████████| 55/55 [00:41<00:00,  1.32it/s]

Фото выгружены


In [441]:
df.head()

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,developer.orgBankruptMsgDttm,newBuildingId,conclusion,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,7.0,52.0,77.0,"г Москва, район Теплый стан, ул Профсоюзная, в...","«ДОМ №128», «ДОМ 128»",21.0,21.0,0.0,2019-04-09,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,14.0,77.0,"д Столбово, д. 2","ЖК ""Москвичка""",2.0,15.0,1553.0,2019-12-12,Панель,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,86.0,50.0,"ГОРОД БАЛАШИХА, УЛИЦА ЛУКИНО, вл. 51","ЖК ""Новая Алексеевская роща""",NaN,NaN,108.0,2018-12-14,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,92.0,77.0,"п Москва, д. 2, корпус 4",NaN,NaN,NaN,1376.0,2019-03-25,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11.0,102.0,77.0,"поселение Сосенское, вблизи д. Столбово, д. 2,...",NaN,NaN,NaN,709.0,2018-12-12,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [442]:
df.shape

(31416, 110)